In [3]:
using JuMP, Cbc, NamedArrays

# Sets of schools (S) and areas (A)
S = 1:3 
A = 1:5

# Number of students in each district
n = Dict(zip(A,[1200, 1000, 1700, 2000, 2500]))

# Capacity of each school
b = Dict(zip(S,[3900, 3100, 2100]))

# Distances between district/school pairs
d_matrix = [2.7 0.5 1.6;
            1.4 0.7 2.0;
            2.4 2.9 0.1;
            1.1 0.8 1.3;
            0.5 1.9 2.2]
d_NA = NamedArray(d_matrix, (A, S), ("district", "school"))

# Fraction of minority students in the district
g = Dict(zip(A,[0.2, 0.1, 0.85, 0.6, 0.9]))

# Minimum and maximum minority fraction in each school
L = 0.25
U = 0.8

# Minimum enrollment in each school
K = 200

# Create the model 
m = Model()

# Define non-negative variables x[a, s] for allocation of 'a' items to 's' slots
@variable(m, x[A, S] >=0)

# Define non-negative variables y[s] representing the total allocated to slot 's'
@variable(m, y[S] >=0)

# Set the objective to minimize: the sum of distances (d_NA[a,s]) times allocations x[a, s]
@objective(m, Min, sum(x[a, s] * d_NA[a,s] for a in A, s in S))

# Constraint: For each slot 's', the sum of allocations x[a,s] must equal y[s]
@constraint(m, def_y[s in S], sum(x[a,s] for a in A) == y[s])

# Constraint: Ensure each item 'a' is fully assigned, summed over all slots 's'
@constraint(m, assign_all[a in A], sum(x[a,s] for s in S) == n[a])

# Constraint: The total allocated to each slot 's' cannot exceed its capacity b[s]
@constraint(m, school_cap[s in S], y[s] <= b[s])

# Constraint: The weighted sum of allocations in each slot 's' must not exceed U times y[s]
@constraint(m, upper_cap[s in S], sum(g[a]*x[a,s] for a in A) <= U*y[s])

# Constraint: The weighted sum of allocations in each slot 's' must at least be L times y[s]
@constraint(m, lower_cap[s in S], sum(g[a]*x[a,s] for a in A) >= L*y[s])

# Constraint: Each slot 's' must have a minimum enrollment of K
@constraint(m, min_enroll[s in S], y[s] >= K)

# Set the solver for the optimization problem, here using Cbc, a linear integer optimizer
set_optimizer(m, Cbc.Optimizer)

# Solve the model
optimize!(m)

# Print the allocation result for x[a, s]
println(value.(x))

# Print the total distance traveled, which is the objective value of the model
println("total distance traveled: ", objective_value(m))

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:5
    Dimension 2, 1:3
And data, a 5×3 Matrix{Float64}:
    0.0               1058.333333333334    141.66666666666615
  116.66666666666553   883.3333333333342     0.0
    0.0                  0.0              1699.9999999999998
  841.6666666666683   1158.333333333332      0.0
 2500.0                  0.0                 0.0
total distance traveled: 4810.0
Presolve 14 (-6) rows, 18 (0) columns and 69 (-6) elements
0  Obj 0 Primal inf 12685.359 (11)
13  Obj 4810
Optimal - objective value 4810
After Postsolve, objective 4810, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 4810 - 13 iterations time 0.002, Presolve 0.00
